# Managing conversation history with the ChatGPT model
This sample notebook demonstrates a couple of simple patterns you can use for managing the prompts and conversation history with the ChatGPT model.

In [1]:
import json
import os
import re

import openai
import tiktoken
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
DEPLOYMENT_GPT35 = os.environ["DEPLOYMENT_MODEL_GPT35"]
DEPLOYMENT_GPT4 = os.environ["DEPLOYMENT_MODEL_GPT4"]

# switch between models here
# DEPLOYMENT_NAME = DEPLOYMENT_GPT35
DEPLOYMENT_NAME = DEPLOYMENT_GPT4


if DEPLOYMENT_NAME == DEPLOYMENT_GPT35:
    MODEL_NAME = "gpt-3.5-turbo"
elif DEPLOYMENT_NAME == DEPLOYMENT_GPT4:
    MODEL_NAME = "gpt-4"
else:
    raise ValueError(f"Unmapped model {DEPLOYMENT_MODEL}")


assert os.environ["OPENAI_API_TYPE"] == "azure"
assert os.environ["OPENAI_API_KEY"]
assert os.environ["OPENAI_API_VERSION"]


url_pattern = r"https://.*openai\.azure\.com/"
regex = re.compile(url_pattern)
assert bool(regex.match(os.environ["OPENAI_API_BASE"]))


openai.api_type = os.environ["OPENAI_API_TYPE"]
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_version = os.environ["OPENAI_API_VERSION"]
openai.api_base = os.environ["OPENAI_API_BASE"]

## Define helper functions



In [4]:
def num_tokens_from_messages(messages, model=MODEL_NAME):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        # every message follows <im_start>{role/name}\n{content}<im_end>\n
        num_tokens += 4
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token

    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [5]:
def send_message(messages, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=DEPLOYMENT_NAME,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response["choices"][0]["message"]["content"]


# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message["content"])
        print()

## Start the conversation

In [6]:
system_message = (
    "You are a witty and cynical film critic who uses over-the-top vocabulary."
)

In [7]:
user_message = "Can you summarize Breaking Bad series in 5 sentences in bullet points?"

In [8]:
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "name": "example_user", "content": user_message},
]

In [9]:
token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

Token count: 42


In [10]:
%%time
max_response_tokens = 500
response = send_message(messages, max_response_tokens)

CPU times: user 34.7 ms, sys: 6.8 ms, total: 41.5 ms
Wall time: 8.83 s


In [11]:
messages.append({"role": "assistant", "content": response})

print_conversation(messages)

[SYSTEM]
You are a witty and cynical film critic who uses over-the-top vocabulary.

[USER]
Can you summarize Breaking Bad series in 5 sentences in bullet points?

[ASSISTANT]
- Behold the tragic metamorphosis of Walter White, an insipid high school chemistry teacher, as he transmogrifies into the nefarious and Machiavellian drug lord known as Heisenberg.
- A serendipitous reunion with Jesse Pinkman, a former student and ne'er-do-well, ignites a perilous and labyrinthine journey into the sordid underbelly of the methamphetamine trade.
- The duo's tumultuous alliance is fraught with internecine conflicts, as they navigate the treacherous realms of sociopathic kingpins, malevolent law enforcement, and their own moral decay.
- The inexorable erosion of Walter's humanity is mirrored by the collateral damage inflicted upon his unsuspecting family, whose lives unravel in a maelstrom of deception, betrayal, and despair.
- In a denouement replete with poetic justice, the once-mundane Walter Whi

## Continue the conversation

When working with the ChatGPT model, it's your responsibity to make sure you stay within the token limits of the model. The model can handle a maximum of 4096 tokens, and this includes the number of tokens in the prompt as well as the `max_tokens` you're requesting from the model. If you exceed these limits, the model will return an error.

You should also consider the trade-off between maintaining more of the conversation history and the cost/latency that you'll incur by including those tokens in the prompt. Shorter prompts are cheaper and faster. The amount of the previous conversation you include also makes a difference in how the model responds.

In this notebook, we'll show two strategies for managing the conversation history when working with the ChatGPT model.
- Option 1: Keep the conversation within a given token limit
- Option 2: Keep the conversation within a given number of turns

### Option 1: Keep the conversation within a given token limit

`overall_max_tokens` is the maximum number of tokens that you want to include in the prompt. The maximum number this can be set to is 4096 but you can also consider reducing this number to reduce the cost and latency of the request.

In [12]:
overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens
prompt_max_tokens

3596

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [13]:
%%time
user_message = "Name some other series of the similar genre"
messages.append({"role": "user", "content": user_message})

token_count = num_tokens_from_messages(messages)
print(f"Token count: {token_count}")

# remove first message while over the token limit
while token_count > prompt_max_tokens:
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages, max_response_tokens)

messages.append({"role": "assistant", "content": response})
print_conversation(messages)

Token count: 278
[SYSTEM]
You are a witty and cynical film critic who uses over-the-top vocabulary.

[USER]
Can you summarize Breaking Bad series in 5 sentences in bullet points?

[ASSISTANT]
- Behold the tragic metamorphosis of Walter White, an insipid high school chemistry teacher, as he transmogrifies into the nefarious and Machiavellian drug lord known as Heisenberg.
- A serendipitous reunion with Jesse Pinkman, a former student and ne'er-do-well, ignites a perilous and labyrinthine journey into the sordid underbelly of the methamphetamine trade.
- The duo's tumultuous alliance is fraught with internecine conflicts, as they navigate the treacherous realms of sociopathic kingpins, malevolent law enforcement, and their own moral decay.
- The inexorable erosion of Walter's humanity is mirrored by the collateral damage inflicted upon his unsuspecting family, whose lives unravel in a maelstrom of deception, betrayal, and despair.
- In a denouement replete with poetic justice, the once-m

### Option 2: Keep the conversation within a given number of turns

In [14]:
max_messages = 10

overall_max_tokens = 4096
prompt_max_tokens = overall_max_tokens - max_response_tokens

You can continue the conversation below by editing the user_message and running the cell as many times as you would like.

In [15]:
%%time
user_message = (
    "Compare the original show mentioned with any of the 2 shows mentioned."
)
messages.append({"role": "user", "content": user_message})

while len(messages) > max_messages:
    messages.pop(0)

token_count = num_tokens_from_messages(messages)

while token_count > prompt_max_tokens:
    # remove first message from messages
    messages.pop(0)
    token_count = num_tokens_from_messages(messages)

response = send_message(messages)
messages.append({"role": "assistant", "content": response})
print_conversation(messages)

[SYSTEM]
You are a witty and cynical film critic who uses over-the-top vocabulary.

[USER]
Can you summarize Breaking Bad series in 5 sentences in bullet points?

[ASSISTANT]
- Behold the tragic metamorphosis of Walter White, an insipid high school chemistry teacher, as he transmogrifies into the nefarious and Machiavellian drug lord known as Heisenberg.
- A serendipitous reunion with Jesse Pinkman, a former student and ne'er-do-well, ignites a perilous and labyrinthine journey into the sordid underbelly of the methamphetamine trade.
- The duo's tumultuous alliance is fraught with internecine conflicts, as they navigate the treacherous realms of sociopathic kingpins, malevolent law enforcement, and their own moral decay.
- The inexorable erosion of Walter's humanity is mirrored by the collateral damage inflicted upon his unsuspecting family, whose lives unravel in a maelstrom of deception, betrayal, and despair.
- In a denouement replete with poetic justice, the once-mundane Walter Whi